In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# setup
import ee, geemap, pandas as pd, geopandas as gpd
from shapely.geometry import Point
from scripts.extract_data import get_wpdx_kenya
ee.Authenticate()
ee.Initialize()


- wpdx data - base

In [3]:
# Load and Sample WPdx
import re

wpdx = get_wpdx_kenya()

wpdx.head()
wpdx.isnull().sum().sort_values(ascending=False)


✅ || wpdx_kenya.csv already exists with 21953 rows, skipping fetch.


fecal_coliform_value       21953
clean_adm4                 21953
scheme_id                  21953
fecal_coliform_presence    21953
rehab_year                 21953
                           ...  
distance_to_tertiary           0
distance_to_town               0
water_point_history            0
days_since_report              0
dataset_title                  0
Length: 73, dtype: int64

In [4]:
wpdx.columns 

Index(['row_id', 'source', 'lat_deg', 'lon_deg', 'report_date', 'status_id',
       'water_source_clean', 'water_source_category', 'water_tech_clean',
       '_water_tech_category', 'facility_type', 'clean_country_name',
       'clean_country_id', 'clean_adm1', 'clean_adm2', 'clean_adm3',
       'clean_adm4', 'install_year', 'installer', 'rehab_year',
       'rehabilitator', 'management_clean', 'status_clean', 'pay_clean',
       'fecal_coliform_presence', 'fecal_coliform_value',
       'subjective_quality_clean', 'activity_id', 'scheme_id', 'wpdx_id',
       'notes', 'orig_lnk', 'photo_lnk', 'country_id', 'data_lnk',
       'distance_to_primary', 'distance_to_secondary', 'distance_to_tertiary',
       'distance_to_city', 'distance_to_town', 'water_point_history',
       'local_population', 'served_population', 'rehab_priority',
       '_pop_who_would_gain_access', 'crucialness', 'pressure', 'usage_cap',
       'days_since_report', 'staleness_score', 'is_latest', 'location_id',
       

In [5]:
# --- Standardize and Rename Columns ---
# No need for the full generic column cleaning with re.sub if you know the exact names.
# However, for robustness, we can still ensure they are lowercased and stripped IF needed.
# For now, let's target the exact names you provided.

# Rename 'wpdx_id' to 'pt_id'
if 'wpdx_id' in wpdx.columns:
    wpdx.rename(columns={'wpdx_id': 'pt_id'}, inplace=True)
else:
    # If for some reason 'wpdx_id' is missing, you'd need to identify the correct ID column.
    raise KeyError("'wpdx_id' column not found. Please verify the actual ID column name.")

# Ensure 'pt_id' is string type
wpdx['pt_id'] = wpdx['pt_id'].astype(str)

# Ensure 'lat_deg' and 'lon_deg' are numeric and rounded
# They are already correctly named, just need conversion and null check.
wpdx['latitude'] = pd.to_numeric(wpdx['lat_deg'], errors='coerce').round(6)
wpdx['longitude'] = pd.to_numeric(wpdx['lon_deg'], errors='coerce').round(6)
wpdx.dropna(subset=['latitude', 'longitude'], inplace=True)


# Filter for Kenya and non-null lat/lon
# Using 'clean_country_name' as per your original intent and its presence.
if 'clean_country_name' in wpdx.columns:
    wpdx = wpdx[(wpdx['clean_country_name'].str.lower() == 'kenya')]
else:
    raise KeyError("'country_name' column not found. Please verify the country column name.")

# Sample 5000 points
DESIRED_SAMPLED_POINTS_COUNT = 5000
if len(wpdx) >= DESIRED_SAMPLED_POINTS_COUNT:
    wpdx = wpdx.sample(n=DESIRED_SAMPLED_POINTS_COUNT, random_state=42).reset_index(drop=True)
else:
    print(f"Warning: Less than {DESIRED_SAMPLED_POINTS_COUNT} points available for sampling. Using all {len(wpdx)} points.")

print(f"Sampled {len(wpdx)} water points for GEE processing.")

# --- Prepare for GEE (select only necessary columns with correct names for geemap) ---
wpdx_for_gee = wpdx[['pt_id', 'latitude', 'longitude']]
# Note: geemap.pandas_to_ee expects 'latitude' and 'longitude' by default if no other names specified.
# So, we map lat_deg/lon_deg to latitude/longitude for clean passing.

Sampled 5000 water points for GEE processing.


In [6]:
wpdx_for_gee.head()

,pt_id,latitude,longitude
0,6GGP5HF3+VR8,0.174668,34.554572
1,6GGP6HC8+85Q,0.220830,34.565483
2,6GGP28VQ+MWR,0.044235,34.339775
3,6GFPHQ2V+2MF,-0.449933,34.794163
4,6GGPFRWJ+VH8,0.497160,34.831463


In [7]:
# Set the END date to June 30th, 2025 
END_DATE = '2025-06-30' 

# Recalculate start dates based on the new END date
START_7_DAYS  = ee.Date(END_DATE).advance(-7, 'day').format('YYYY-MM-dd').getInfo()
START_30_DAYS = ee.Date(END_DATE).advance(-30, 'day').format('YYYY-MM-dd').getInfo()
START_90_DAYS = ee.Date(END_DATE).advance(-90, 'day').format('YYYY-MM-dd').getInfo()

print(f"Date range for composites: {START_90_DAYS} to {END_DATE}")
print(f"Calculated START_7: {START_7_DAYS}")
print(f"Calculated START_30: {START_30_DAYS}")
print(f"Calculated START_90: {START_90_DAYS}")

Date range for composites: 2025-04-01 to 2025-06-30
Calculated START_7: 2025-06-23
Calculated START_30: 2025-05-31
Calculated START_90: 2025-04-01


In [8]:
# Define the image collections
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").select("precipitation")
ndvi = ee.ImageCollection("MODIS/061/MOD13Q1").select("NDVI")
lst = ee.ImageCollection("MODIS/061/MOD11A2").select("LST_Day_1km")
soil = ee.ImageCollection("NASA_USDA/HSL/SMAP_soil_moisture").select("ssm")
pop = ee.Image("WorldPop/GP/100m/pop/KEN_2020").select("population") # 'pop' band in WorldPop 100m dataset is 'population'

C:\Users\Lewis\AppData\Roaming\Python\Python312\site-packages\ee\deprecation.py:209: DeprecationWarning: 

Attention required for NASA_USDA/HSL/SMAP_soil_moisture! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/NASA_USDA_HSL_SMAP_soil_moisture

  warnings.warn(warning, category=DeprecationWarning)


In [9]:
features = []
for _, row in wpdx_for_gee.iterrows():
    geom = ee.Geometry.Point([row['longitude'], row['latitude']])
    properties = row.drop(['latitude', 'longitude']).to_dict()
    features.append(ee.Feature(geom, properties))

wpdx_fc = ee.FeatureCollection(features)
print(f"Converted {wpdx_fc.size().getInfo()} water points to ee.FeatureCollection.")

Converted 5000 water points to ee.FeatureCollection.


In [10]:
# Function for Extraction 


def extract_var(ic_or_img, start, end, band, scale):
    """
    Extracts mean value of a band for each feature in wpdx_fc from an ImageCollection or Image.
    Applies scale factors and handles static images.
    """
    if isinstance(ic_or_img, ee.ImageCollection):
        img = ic_or_img.filterDate(start, end).mean().select(band)
    else: # Assume it's an ee.Image for static data
        img = ic_or_img.select(band)

    # Apply specific scale factors/conversions
    if band == "NDVI":
        img = img.multiply(0.0001) # MODIS NDVI scale factor
    elif band == "LST_Day_1km":
        img = img.multiply(0.02).subtract(273.15) # MODIS LST scale factor (K to C)

    # Sample regions and return as FeatureCollection
    return img.sampleRegions(collection=wpdx_fc, scale=scale, geometries=True)

print("Extraction function defined.")


Extraction function defined.


In [11]:
# --- 4. Apply for All Variables ---

# CHIRPS (Daily Precipitation)
chirps_7_fc = extract_var(chirps, START_7_DAYS, END_DATE, "precipitation", 5000)
chirps_30_fc = extract_var(chirps, START_30_DAYS, END_DATE, "precipitation", 5000)
chirps_90_fc = extract_var(chirps, START_90_DAYS, END_DATE, "precipitation", 5000)

# NDVI (16-day composite)
ndvi_7_fc = extract_var(ndvi, START_7_DAYS, END_DATE, "NDVI", 250)
ndvi_30_fc = extract_var(ndvi, START_30_DAYS, END_DATE, "NDVI", 250)
ndvi_90_fc = extract_var(ndvi, START_90_DAYS, END_DATE, "NDVI", 250)

# LST (8-day composite)
lst_7_fc = extract_var(lst, START_7_DAYS, END_DATE, "LST_Day_1km", 1000)
lst_30_fc = extract_var(lst, START_30_DAYS, END_DATE, "LST_Day_1km", 1000)
lst_90_fc = extract_var(lst, START_90_DAYS, END_DATE, "LST_Day_1km", 1000)

# Soil Moisture (Daily)
soil_7_fc = extract_var(soil, START_7_DAYS, END_DATE, "ssm", 9000)
soil_30_fc = extract_var(soil, START_30_DAYS, END_DATE, "ssm", 9000)
soil_90_fc = extract_var(soil, START_90_DAYS, END_DATE, "ssm", 9000)

# Population Density (Static Image)
pop_feat_fc = extract_var(pop, None, None, "population", 100)

print("Environmental FeatureCollections generated.")

Environmental FeatureCollections generated.


In [12]:
# --- 5. Example of Accessing Data (Optional) ---
try:
    print("\n--- Example Retrieved Data (first feature) ---")
    print(f"CHIRPS 30-day (precipitation): {chirps_30_fc.first().getInfo().get('properties')}")
    print(f"NDVI 30-day (NDVI): {ndvi_30_fc.first().getInfo().get('properties')}")
    print(f"LST 30-day (LST_Day_1km): {lst_30_fc.first().getInfo().get('properties')}")
    print(f"Soil 30-day (ssm): {soil_30_fc.first().getInfo().get('properties')}")
    print(f"Population (population): {pop_feat_fc.first().getInfo().get('properties')}")
except Exception as e:
    print(f"\nCould not retrieve example data (might be empty FeatureCollection or other GEE error): {e}")



--- Example Retrieved Data (first feature) ---
CHIRPS 30-day (precipitation): {'precipitation': 5.8042988777160645, 'pt_id': '6GGP5HF3+VR8'}
NDVI 30-day (NDVI): {'NDVI': 0.7792, 'pt_id': '6GGP5HF3+VR8'}
LST 30-day (LST_Day_1km): {'LST_Day_1km': 23.620000000000005, 'pt_id': '6GGP5HF3+VR8'}

Could not retrieve example data (might be empty FeatureCollection or other GEE error): Image.select: Band pattern 'ssm' was applied to an Image with no bands. See https://developers.google.com/earth-engine/guides/debugging#no-bands


- the soil one needs to be removed maybe

In [13]:
# --- 5. Example of Accessing Data (Optional) ---
try:
    print("\n--- Example Retrieved Data (first feature) ---")
    print(f"CHIRPS 30-day (precipitation): {chirps_30_fc.first().getInfo().get('properties')}")
    print(f"NDVI 30-day (NDVI): {ndvi_30_fc.first().getInfo().get('properties')}")
    print(f"LST 30-day (LST_Day_1km): {lst_30_fc.first().getInfo().get('properties')}")
    print(f"Population (population): {pop_feat_fc.first().getInfo().get('properties')}")
except Exception as e:
    print(f"\nCould not retrieve example data (might be empty FeatureCollection or other GEE error): {e}")



--- Example Retrieved Data (first feature) ---
CHIRPS 30-day (precipitation): {'precipitation': 5.8042988777160645, 'pt_id': '6GGP5HF3+VR8'}
NDVI 30-day (NDVI): {'NDVI': 0.7792, 'pt_id': '6GGP5HF3+VR8'}
LST 30-day (LST_Day_1km): {'LST_Day_1km': 23.620000000000005, 'pt_id': '6GGP5HF3+VR8'}
Population (population): {'population': 10.462027549743652, 'pt_id': '6GGP5HF3+VR8'}


## Convert to Pandas Dataframes

In [14]:
from datetime import datetime

df_chirps_7 = geemap.ee_to_df(chirps_7_fc)
df_chirps_30 = geemap.ee_to_df(chirps_30_fc)
df_chirps_90 = geemap.ee_to_df(chirps_90_fc)

df_ndvi_7 = geemap.ee_to_df(ndvi_7_fc)
df_ndvi_30 = geemap.ee_to_df(ndvi_30_fc)
df_ndvi_90 = geemap.ee_to_df(ndvi_90_fc)

df_lst_7 = geemap.ee_to_df(lst_7_fc)
df_lst_30 = geemap.ee_to_df(lst_30_fc)
df_lst_90 = geemap.ee_to_df(lst_90_fc)

df_pop = geemap.ee_to_df(pop_feat_fc)

final_df = wpdx.copy()

def merge_ee_df(main_df, ee_df, new_col_prefix):
    data_cols = [col for col in ee_df.columns if col not in ['pt_id', 'system:index', '.geo']]
    if not data_cols:
        return main_df
    extracted_col_name = data_cols[0]
    ee_df_renamed = ee_df[['pt_id', extracted_col_name]].rename(columns={extracted_col_name: f'{new_col_prefix}_{extracted_col_name}'})
    return pd.merge(main_df, ee_df_renamed, on='pt_id', how='left')

final_df = merge_ee_df(final_df, df_chirps_7, 'chirps_7')
final_df = merge_ee_df(final_df, df_chirps_30, 'chirps_30')
final_df = merge_ee_df(final_df, df_chirps_90, 'chirps_90')

final_df = merge_ee_df(final_df, df_ndvi_7, 'ndvi_7')
final_df = merge_ee_df(final_df, df_ndvi_30, 'ndvi_30')
final_df = merge_ee_df(final_df, df_ndvi_90, 'ndvi_90')

final_df = merge_ee_df(final_df, df_lst_7, 'lst_7')
final_df = merge_ee_df(final_df, df_lst_30, 'lst_30')
final_df = merge_ee_df(final_df, df_lst_90, 'lst_90')

final_df = merge_ee_df(final_df, df_pop, 'pop')

output_csv_filename = f"../data/processed/kenya_waterpoints_environmental_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
final_df.to_csv(output_csv_filename, index=False)

In [15]:
final_df.head()

,row_id,source,lat_deg,lon_deg,report_date,status_id,water_source_clean,water_source_category,water_tech_clean,_water_tech_category,...,chirps_7_precipitation,chirps_30_precipitation,chirps_90_precipitation,ndvi_7_NDVI,ndvi_30_NDVI,ndvi_90_NDVI,lst_7_LST_Day_1km,lst_30_LST_Day_1km,lst_90_LST_Day_1km,pop_population
0,215873,Evidence Action,0.174668,34.554572,2023-02-11T00:00:00.000,Yes,Protected Well,Well,Motorized Pump - Electric,Motorized Pump,...,4.914062,5.804299,11.215961,0.7685,0.77920,0.765850,23.35,23.620,24.402727,10.462028
1,215240,Evidence Action,0.220830,34.565483,2023-11-15T00:00:00.000,Yes,Protected Well,Well,Motorized Pump - Electric,Motorized Pump,...,3.862696,5.864281,11.359583,0.7642,0.77615,0.745817,23.63,23.370,24.260909,6.315442
2,194605,Evidence Action,0.044235,34.339775,2023-11-15T00:00:00.000,Yes,Protected Well,Well,Motorized Pump - Electric,Motorized Pump,...,2.113567,2.798805,7.549499,0.7348,0.73455,0.716650,24.89,24.350,24.922000,5.745561
3,219017,Evidence Action,-0.449933,34.794163,2023-11-17T00:00:00.000,Yes,Borehole/Tubewell,Well,NaN,NaN,...,2.045830,2.790391,7.499787,0.7235,0.73420,0.730017,24.41,24.310,24.433636,3.103306
4,587740,USAID/KEA KIWASH,0.497160,34.831463,2021-02-05T00:00:00.000,No,Undefined Well,Well,Public Tapstand,Public Tapstand,...,4.075452,5.655031,9.077165,0.7256,0.74335,0.711867,23.79,23.435,24.326000,7.758868


In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21368 entries, 0 to 21367
Data columns (total 85 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   row_id                      21368 non-null  int64  
 1   source                      21368 non-null  object 
 2   lat_deg                     21368 non-null  float64
 3   lon_deg                     21368 non-null  float64
 4   report_date                 21368 non-null  object 
 5   status_id                   21368 non-null  object 
 6   water_source_clean          21342 non-null  object 
 7   water_source_category       21342 non-null  object 
 8   water_tech_clean            15937 non-null  object 
 9   _water_tech_category        15787 non-null  object 
 10  facility_type               21368 non-null  object 
 11  clean_country_name          21368 non-null  object 
 12  clean_country_id            21368 non-null  object 
 13  clean_adm1                  213